In [71]:
import moon.data
import moon.problem

In [72]:
probs = moon.data.read_problems('data/cleaned_probs.csv')

In [73]:
import numpy as np

x = np.array([p.array_3d for p in probs])
y = np.array([p.grade.ordinal for p in probs])

print(x.shape, y.shape)

(30991, 18, 11, 3) (30991, 17)


In [74]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.keras.backend as K

class OrdinalAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name='ordinal_acc_within_k', k=0, threshold=0.5, **kwargs):
        super(OrdinalAccuracy, self).__init__(name=name, **kwargs)
        self.k = k
        self.threshold = threshold
        self.total = self.add_weight(name='total', initializer='zeros')
        self.count = self.add_weight(name='count', initializer='zeros')
    
    def update_state(self, y_true, y_pred, sample_weight=None):
        # See section 2.3: http://calla.rnet.missouri.edu/cheng_courses/rank.pdf
        pred_thresh = tf.cast(y_pred >= self.threshold, 'int8')
        true_thresh = tf.cast(y_true >= self.threshold, 'int8')
        pred_idx = tf.argmin(pred_thresh, axis=-1) - 1
        true_idx = tf.argmin(true_thresh, axis=-1) - 1
        distance_bools = (tf.abs(pred_idx - true_idx) <= self.k)
        correct = tf.reduce_sum(tf.cast(distance_bools, 'float32'))
        incorrect = tf.reduce_sum(tf.cast(tf.logical_not(distance_bools), 'float32'))
        self.total.assign_add(correct)
        self.count.assign_add(correct + incorrect)
    
    def result(self):
        return self.total / self.count

    
# α = 0.3
p = .4
input_shape = (*moon.problem.Problem.GRID_SHAPE, 3)
hiddens = [256, 128, 64]
hidden_activation = 'swish'
output_shape = moon.problem.Grade.N_GRADES
output_activation = 'sigmoid'
loss = 'mse'
adam_lr = 1e-3

metrics = [OrdinalAccuracy(name='acc0', k=0), OrdinalAccuracy(name='acc1', k=1), OrdinalAccuracy(name='acc2', k=2)]
optim = tf.keras.optimizers.Adam(lr=adam_lr)

In [75]:
in_x = layers.Input(shape=input_shape)
features = layers.Flatten()(in_x)
for nodes in hiddens:
    features = layers.Dense(nodes, activation=hidden_activation)(features)
    # features = layers.ReLU(α)(features)
    if p > 0: features = layers.Dropout(p)(features)
out_y = layers.Dense(output_shape, activation=output_activation)(features)

model = tf.keras.Model(in_x, out_y, name='Feedforward_classifier')
model.compile(loss=loss, optimizer=optim, metrics=metrics)
# model.summary()

In [76]:
split = 0.2
indices = np.arange(len(probs))
np.random.shuffle(indices)
n_test = int(len(probs) * split)

x_train, y_train = x[indices[n_test:]], y[indices[n_test:]]
x_test, y_test = x[indices[0:n_test]], y[indices[0:n_test]]
print(x_train.shape, x_test.shape)

(24793, 18, 11, 3) (6198, 18, 11, 3)


In [77]:
callbacks = [tf.keras.callbacks.EarlyStopping(patience=4, verbose=1)]
batch_size = 64
max_epochs = 200

history = model.fit(x_train, y_train, batch_size=batch_size, validation_data=(x_test, y_test), epochs=max_epochs, callbacks=callbacks)

# decreasing, no p, flattened, 100e

# decreasing, p=.4, flattened, 28e
# loss: 0.0396 - acc0: 0.4144 - acc1: 0.7597 - acc2: 0.9229 - val_loss: 0.0414 - val_acc0: 0.3925 - val_acc1: 0.7375 - val_acc2: 0.9140

# decreasing, p=.4, 3d, 24e
# loss: 0.0409 - acc0: 0.4075 - acc1: 0.7475 - acc2: 0.9123 - val_loss: 0.0421 - val_acc0: 0.3945 - val_acc1: 0.7389 - val_acc2: 0.9076

Epoch 1/200
388/388 [==============================] - 2s 5ms/step - loss: 0.0670 - acc0: 0.2926 - acc1: 0.6149 - acc2: 0.8081 - val_loss: 0.0467 - val_acc0: 0.3598 - val_acc1: 0.6978 - val_acc2: 0.8833
Epoch 2/200
388/388 [==============================] - 1s 4ms/step - loss: 0.0489 - acc0: 0.3587 - acc1: 0.6829 - acc2: 0.8687 - val_loss: 0.0461 - val_acc0: 0.3787 - val_acc1: 0.6996 - val_acc2: 0.8837
Epoch 3/200
388/388 [==============================] - 2s 4ms/step - loss: 0.0474 - acc0: 0.3665 - acc1: 0.6936 - acc2: 0.8761 - val_loss: 0.0451 - val_acc0: 0.3666 - val_acc1: 0.7068 - val_acc2: 0.8924
Epoch 4/200
388/388 [==============================] - 1s 4ms/step - loss: 0.0459 - acc0: 0.3791 - acc1: 0.7044 - acc2: 0.8828 - val_loss: 0.0437 - val_acc0: 0.3838 - val_acc1: 0.7231 - val_acc2: 0.9006
Epoch 5/200
388/388 [==============================] - 2s 4ms/step - loss: 0.0445 - acc0: 0.3868 - acc1: 0.7125 - acc2: 0.8940 - val_loss: 0.0433 - val_acc0: 0.3819 - val_acc1: 0.7298 - va

In [78]:
# print(np.round(model.predict(x_test[99][np.newaxis, :]) * 100))
# print(y_test[99])
y_true = y_test[[2, 4, 8, 16, 32]]
y_pred = model.predict(x_test[[2, 4, 8, 16, 32]])

In [79]:
pred_thresh = tf.cast(y_pred >= .5, 'int8')
true_thresh = tf.cast(y_true >= .5, 'int8')
print(pred_thresh, true_thresh)

# Since tf.argmin returns the lowest index possible, this gets us the index of the last True value.
# Note that this is -1 if the network doesn't many any prediction at all.
pred_idx = tf.argmin(pred_thresh, axis=-1) - 1
true_idx = tf.argmin(true_thresh, axis=-1) - 1
print(pred_idx, true_idx)

distance_bools = tf.abs(pred_idx - true_idx) <= 0
correct = tf.reduce_sum(tf.cast(distance_bools, 'float32'))
incorrect = tf.reduce_sum(tf.cast(tf.logical_not(distance_bools), 'float32'))
print(distance_bools, correct, incorrect)

print(correct / (correct+incorrect))

tf.Tensor(
[[1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]], shape=(5, 17), dtype=int8) tf.Tensor(
[[1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]], shape=(5, 17), dtype=int8)
tf.Tensor([7 8 7 8 6], shape=(5,), dtype=int64) tf.Tensor([8 9 6 7 6], shape=(5,), dtype=int64)
tf.Tensor([False False False False  True], shape=(5,), dtype=bool) tf.Tensor(1.0, shape=(), dtype=float32) tf.Tensor(4.0, shape=(), dtype=float32)
tf.Tensor(0.2, shape=(), dtype=float32)
